In [43]:
from google.oauth2.service_account import Credentials
import gspread
from googleapiclient.discovery import build
import time
from datetime import datetime

import pandas as pd
# 출력물의 생략없이 모두 출력
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
np.set_printoptions(threshold=np.inf)


from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import requests
import json

# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
# font_path = "D:/project/fonts/NanumGothic.ttf"
# font_path = 'G:/다른 컴퓨터/My_desktop/project/fonts/NanumGothic.ttf'
# 한글 폰트 설정
font_path = 'G:/다른 컴퓨터/내 컴퓨터/project/fonts/NanumGothic.ttf'
# font_path = 'C:/project/fonts/NanumGothic.ttf'/
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

In [44]:
# 스코프 설정
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# 서비스 계정 키 불러오기
credentials = None
credentials = Credentials.from_service_account_file(
    'G:/.shortcut-targets-by-id/1pm-mzTMxN90n5Axi5ATZbh2kl8wZEOlv/2023 공공SW 수발주자 역량강화 교육 프로젝트/2.교육운영/설문조사/만족도 조사/오프라인/driven-current-398407-9f03dd3a7cc6.json', scopes=scopes)

# Google Drive API 클라이언트 생성
drive_service = build('drive', 'v3', credentials=credentials)

# Google Sheets API 클라이언트 생성
client = gspread.authorize(credentials)

# 폴더 ID로 폴더 내의 모든 파일 정보 가져오기 (폴더 ID는 Google Drive URL에서 확인 가능)
folder_id = "1XEZ8bMC7lOQ30yhKCR5Jsr9SQ-WGKpge"
query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet'"
results = drive_service.files().list(q=query).execute()
file_list = results.get('files', [])

# 성공 카운터 및 데이터프레임 리스트 초기화
success_count = 0
dfs = []

# 폴더 내의 모든 Google Sheets 불러오기
for file in file_list:
    try:
        # 파일명에서 '추출할 내용' 파싱
        file_name = file['name']
        parts = file_name.split('_')  # 언더바를 기준으로 문자열을 나눕니다.
        if len(parts) > 1:
            extracted_content = parts[1]  # 두 번째 섹션(즉, '추출할 내용')을 가져옵니다.
        else:
            extracted_content = None  # 만약 언더바로 구분된 섹션이 없다면 None을 할당합니다.

        sheet = client.open(file_name).sheet1
        data = sheet.get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0])

        # '신청기관'이라는 새로운 컬럼을 생성하고, 파싱한 '추출할 내용'을 할당합니다.
        df['신청기관'] = extracted_content

        dfs.append(df)
        success_count += 1
        time.sleep(2)  # API 호출 빈도를 제어하기 위해 각 호출 사이에 2초 지연시간을 설정합니다.
    except Exception as e:
        print(f"An error occurred with file {file['name']}: {e}")

# 성공 카운트 출력
print(f"Successfully loaded {success_count} out of {len(file_list)} files.")

Successfully loaded 15 out of 15 files.


In [45]:
for df, file in zip(dfs, file_list):
    print(f"{file['name']}의 shape: {df.shape}")

# 성공 카운트 출력
print(f"Successfully loaded {success_count} out of {len(file_list)} files.")


0919_KOTRA_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (25, 20)
0918_KOTRA_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (25, 20)
1208_강원특별자치도청_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (34, 20)
1207_수원시청_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (25, 20)
1103_국민건강보험공단_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (19, 20)
1114_인천광역시교육청_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (8, 20)
1102_국민건강보험공단_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (23, 20)
1102_KOICA_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (9, 20)
1101_국민건강보험공단_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (23, 20)
1031_국민건강보험공단_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (24, 20)
1030_국민건강보험공단_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (12, 20)
1012_산업은행_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (16, 20)
0914_한국교육개발원_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (42, 20)
1005_국민연금공단_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (53, 20)
0920_한국문화정보원_공공SW사업 방문 교육 만족도 조사(응답)의 shape: (13, 20)
Successfully loaded 15 out of 15 files.


In [46]:
for df, file in zip(dfs, file_list):
    # 파일명에서 날짜 부분 추출
    date_str = file['name'].split('_')[0]
    
    # 날짜를 datetime 형태로 변환
    # 여기서는 2023년을 기준으로 합니다. 다른 연도를 원하시면 그에 맞게 조정하세요.
    lecture_date = datetime.strptime(f"2023{date_str}", '%Y%m%d')
    
    # '강의일' 컬럼 생성 및 값 적용
    df['강의일'] = lecture_date

In [47]:
# final_dfs는 여러 개의 DataFrame을 포함하는 리스트입니다.
df = pd.concat(dfs, ignore_index=True)


In [48]:
df['수강하신 강의명을 체크해주세요.'].unique()

array(['', '공공SW사업 과업변경 실무',
       '공공SW사업 감사사례, 공공SW사업 과업변경 실무, 공공SW사업 IT 신기술 트렌드',
       '공공SW사업 발주 프로세스 및 법제도 이해, 공공SW사업 사업 기획(예산 및 ISP) 수립, 공공SW사업 제안요청서 상세 요구사항 작성, 공공SW사업 평가 선정 계약 및 관리감독',
       '공공SW사업 발주 프로세스 및 법제도 이해, 공공SW사업 제안요청서 상세 요구사항 작성',
       '공공SW사업 제안요청서 상세 요구사항 작성', '공공SW사업 발주 프로세스 및 법제도 이해',
       '소프트웨어 사업대가 및 기능점수(FP) 산정 교육 (특화)', '공공SW사업 감사사례',
       '공공SW사업 발주 프로세스 및 법제도 이해, 공공SW사업 사업기획(예산 및 ISP) 수립, 공공SW사업 제안요청서 상세 요구사항 작성',
       '공공SW사업 발주 프로세스 및 법제도 이해, 공공SW사업 신기술동향',
       '공공SW사업 사업 기획(예산 및 ISP) 수립, 공공SW사업 제안요청서 상세 요구사항 작성',
       '공공SW사업 사업 기획(예산 및 ISP) 수립', '공공SW사업 평가‧선정‧계약 및 관리감독',
       '공공SW사업 사업 기획(예산 및 ISP) 수립, 공공SW사업 제안요청서 상세 요구사항 작성, 공공SW사업 발주 프로세스 및 법제도 이해, 공공SW사업 평가‧선정‧계약 및 관리감독',
       '공공SW사업 제안요청서 상세 요구사항 작성, 공공SW사업 감사사례, 공공SW사업 과업변경 실무, 공공SW사업 신기술 동향'],
      dtype=object)

In [49]:
df.head()

,타임스탬프,이메일 주소,성함을 입력해주세요.,소속기관의 유형을 체크해주세요.,소속기관명을 입력해주세요.,소속 부서를 입력해주세요.,수강하신 강의명을 체크해주세요.,1-1. 강의에 대해 전체적으로 만족하였다.,1-2. 교재는 강의 내용을 이해하는데 도움이 되도록 만들어졌다.,1-3. 전반적으로 교육 내용이 도움이 되었다.,1-4. 본 강의를 다른 사람에게도 추천할 의향이 있다.,1-5. 기타 의견(없으시면 생략),2-1. 강의 준비 및 열의에 만족하였다.,2-2. 강의 기법과 질문 응답 등에 만족하였다.,2-3. 강의에 대한 강사의 전문성에 만족하였다.,2-4. 기타 의견(없으시면 생략),"3-1. 환경, 조명, 소음, 화면공유 등 교육 진행 환경이 만족스러웠다.",3-2. 교육운영 인력의 교육 진행과 응대는 만족스러웠다.,3-3. 기타 의견(없으시면 생략),신청기관,강의일
0,2023. 10. 4 오전 5:58:09,215002,구현모,공공기관,KOTRA,고객전략팀,,7,7,7,7,,7,7,7,,7,7,,KOTRA,2023-09-19
1,2023. 9. 21 오후 2:31:28,meekim@kotra.or.kr,김미연,공공기관,KOTRA,정보보안운영팀,공공SW사업 과업변경 실무,7,7,7,7,,7,7,7,,7,7,,KOTRA,2023-09-19
2,2023. 9. 22 오후 5:05:14,206040,김성재,공공기관,KOTRA,해외정보관리팀,,6,6,6,6,,5,5,5,,7,7,,KOTRA,2023-09-19
3,2023. 10. 6 오전 1:41:15,220021,김윤근,공공기관,KOTRA,정보보안운영팀,,6,6,7,6,,6,6,7,,6,7,,KOTRA,2023-09-19
4,2023. 10. 5 오전 11:06:53,215075,김재경,공공기관,KOTRA,정보보안운영팀,,6,6,6,6,,6,6,6,,7,7,,KOTRA,2023-09-19


In [50]:
df.shape

(351, 21)

In [51]:
df.columns

Index(['타임스탬프', '이메일 주소', '성함을 입력해주세요.', '소속기관의 유형을 체크해주세요.', '소속기관명을 입력해주세요.',
       '소속 부서를 입력해주세요.', '수강하신 강의명을 체크해주세요.', '1-1. 강의에 대해 전체적으로 만족하였다.',
       '1-2. 교재는 강의 내용을 이해하는데 도움이 되도록 만들어졌다.', '1-3. 전반적으로 교육 내용이 도움이 되었다.',
       '1-4. 본 강의를 다른 사람에게도 추천할 의향이 있다.', '1-5. 기타 의견(없으시면 생략)',
       '2-1. 강의 준비 및 열의에 만족하였다.', '2-2. 강의 기법과 질문 응답 등에 만족하였다.',
       '2-3. 강의에 대한 강사의 전문성에 만족하였다.', '2-4. 기타 의견(없으시면 생략)',
       '3-1. 환경, 조명, 소음, 화면공유 등 교육 진행 환경이 만족스러웠다.',
       '3-2. 교육운영 인력의 교육 진행과 응대는 만족스러웠다.', '3-3. 기타 의견(없으시면 생략)', '신청기관',
       '강의일'],
      dtype='object')

In [52]:
# 설문 컬럼들
columns_to_convert_int = [
    '1-1. 강의에 대해 전체적으로 만족하였다.',
    '1-2. 교재는 강의 내용을 이해하는데 도움이 되도록 만들어졌다.',
    '1-3. 전반적으로 교육 내용이 도움이 되었다.',
    '1-4. 본 강의를 다른 사람에게도 추천할 의향이 있다.',
    '2-1. 강의 준비 및 열의에 만족하였다.',
    '2-2. 강의 기법과 질문 응답 등에 만족하였다.',
    '2-3. 강의에 대한 강사의 전문성에 만족하였다.',
    '3-1. 환경, 조명, 소음, 화면공유 등 교육 진행 환경이 만족스러웠다.',
    '3-2. 교육운영 인력의 교육 진행과 응대는 만족스러웠다.'
]

for col in columns_to_convert_int:
    df[col] = df[col].astype(int)

In [53]:
# 새로운 컬럼을 생성
df['1-1'] = df['1-1. 강의에 대해 전체적으로 만족하였다.'] / 7 * 100
df['1-2'] = df['1-2. 교재는 강의 내용을 이해하는데 도움이 되도록 만들어졌다.'] / 7 * 100
df['1-3'] = df['1-3. 전반적으로 교육 내용이 도움이 되었다.'] / 7 * 100
df['1-4'] = df['1-4. 본 강의를 다른 사람에게도 추천할 의향이 있다.'] / 7 * 100
df['2-1'] = df['2-1. 강의 준비 및 열의에 만족하였다.'] / 7 * 100
df['2-2'] = df['2-2. 강의 기법과 질문 응답 등에 만족하였다.'] / 7 * 100
df['2-3'] = df['2-3. 강의에 대한 강사의 전문성에 만족하였다.'] / 7 * 100
df['3-1'] = df['3-1. 환경, 조명, 소음, 화면공유 등 교육 진행 환경이 만족스러웠다.'] / 7 * 100
df['3-2'] = df['3-2. 교육운영 인력의 교육 진행과 응대는 만족스러웠다.'] / 7 * 100


In [54]:
평균을_계산할_컬럼들 = ['1-1', '1-2', '1-3', '1-4', '2-1', '2-2', '2-3', '3-1', '3-2']

# 각 행의 평균을 계산하고 '평균' 컬럼에 추가합니다.
df['평균'] = df[평균을_계산할_컬럼들].mean(axis=1)
df['평균'] = df['평균'].round(1)


In [55]:
# '강의일', '신청기관', '강좌명'를 기준으로 그룹을 만들고, 해당 그룹의 개수(count)를 계산합니다.
grouped_count_df = df.groupby(['강의일', '신청기관']).size().reset_index(name='new_설문인원')

# 원래 데이터프레임(df)에 업데이트할 '설문인원' 값을 저장할 새로운 컬럼을 생성합니다.
df['new_설문인원'] = None

# 원래 데이터프레임(df)를 순회하면서 '설문인원' 값을 업데이트합니다.
for idx, row in df.iterrows():
    matching_row = grouped_count_df[
        (grouped_count_df['강의일'] == row['강의일']) & 
        (grouped_count_df['신청기관'] == row['신청기관'])
    ]
    if not matching_row.empty:
        df.at[idx, 'new_설문인원'] = matching_row['new_설문인원'].values[0]

# 업데이트된 '설문인원' 값을 확인합니다.
df['설문인원'] = df['new_설문인원']
# 'new_만족도'와 'new_설문인원' 컬럼을 삭제합니다.
df.drop(['new_설문인원'], axis=1, inplace=True)
df[['강의일', '신청기관', '설문인원']].head()

,강의일,신청기관,설문인원
0,2023-09-19,KOTRA,25
1,2023-09-19,KOTRA,25
2,2023-09-19,KOTRA,25
3,2023-09-19,KOTRA,25
4,2023-09-19,KOTRA,25


In [56]:
# '강의일', '신청기관'를 기준으로 그룹을 만들고, '평균' 컬럼의 평균을 계산합니다.
grouped_df = df.groupby(['강의일', '신청기관'])['평균'].mean().reset_index()

# 원래 데이터프레임(df)에 업데이트할 '만족도' 값을 저장할 새로운 컬럼을 생성합니다.
df['new_만족도'] = None

# 원래 데이터프레임(df)를 순회하면서 '만족도' 값을 업데이트합니다.
for idx, row in df.iterrows():
    matching_row = grouped_df[
        (grouped_df['강의일'] == row['강의일']) & 
        (grouped_df['신청기관'] == row['신청기관'])
    ]
    if not matching_row.empty:
        df.at[idx, 'new_만족도'] = matching_row['평균'].values[0]
        

# 업데이트된 '만족도' 값을 확인합니다.
df['만족도'] = df['new_만족도']
df.drop(['new_만족도'], axis=1, inplace=True)
df[['강의일', '신청기관', '평균', '만족도']].head()

,강의일,신청기관,평균,만족도
0,2023-09-19,KOTRA,100.0,95.428
1,2023-09-19,KOTRA,100.0,95.428
2,2023-09-19,KOTRA,84.1,95.428
3,2023-09-19,KOTRA,90.5,95.428
4,2023-09-19,KOTRA,88.9,95.428


In [57]:
df.columns

Index(['타임스탬프', '이메일 주소', '성함을 입력해주세요.', '소속기관의 유형을 체크해주세요.', '소속기관명을 입력해주세요.',
       '소속 부서를 입력해주세요.', '수강하신 강의명을 체크해주세요.', '1-1. 강의에 대해 전체적으로 만족하였다.',
       '1-2. 교재는 강의 내용을 이해하는데 도움이 되도록 만들어졌다.', '1-3. 전반적으로 교육 내용이 도움이 되었다.',
       '1-4. 본 강의를 다른 사람에게도 추천할 의향이 있다.', '1-5. 기타 의견(없으시면 생략)',
       '2-1. 강의 준비 및 열의에 만족하였다.', '2-2. 강의 기법과 질문 응답 등에 만족하였다.',
       '2-3. 강의에 대한 강사의 전문성에 만족하였다.', '2-4. 기타 의견(없으시면 생략)',
       '3-1. 환경, 조명, 소음, 화면공유 등 교육 진행 환경이 만족스러웠다.',
       '3-2. 교육운영 인력의 교육 진행과 응대는 만족스러웠다.', '3-3. 기타 의견(없으시면 생략)', '신청기관',
       '강의일', '1-1', '1-2', '1-3', '1-4', '2-1', '2-2', '2-3', '3-1', '3-2',
       '평균', '설문인원', '만족도'],
      dtype='object')

In [58]:
# '강의일', '신청기관', '강좌명'을 기준으로 '접수인원', '수강인원', '설문인원', '만족도'의 평균을 계산합니다.
# 이후에 '강사' 정보를 추가하기 위해 첫 번째 '강사' 값을 선택합니다.
daily_df = df.groupby(['강의일', '신청기관']).agg({
    '설문인원': 'mean',
    '만족도': 'mean',
}).reset_index()

# '만족도' 컬럼의 값을 float 형식으로 변환
daily_df['만족도'] = daily_df['만족도'].astype(float)

# .round(1) 함수 적용
daily_df['만족도'] = daily_df['만족도'].round(1)

new_columns_order = ['강의일', '신청기관','만족도']
daily_df = daily_df[new_columns_order]

In [59]:
# 결과를 확인합니다.
daily_df

,강의일,신청기관,만족도
0,2023-09-14,한국교육개발원,90.1
1,2023-09-18,KOTRA,94.2
2,2023-09-19,KOTRA,95.4
3,2023-09-20,한국문화정보원,95.0
4,2023-10-05,국민연금공단,95.1
5,2023-10-12,산업은행,95.3
6,2023-10-30,국민건강보험공단,90.1
7,2023-10-31,국민건강보험공단,87.0
8,2023-11-01,국민건강보험공단,94.4
9,2023-11-02,KOICA,91.0


In [60]:
df.head(3)

,타임스탬프,이메일 주소,성함을 입력해주세요.,소속기관의 유형을 체크해주세요.,소속기관명을 입력해주세요.,소속 부서를 입력해주세요.,수강하신 강의명을 체크해주세요.,1-1. 강의에 대해 전체적으로 만족하였다.,1-2. 교재는 강의 내용을 이해하는데 도움이 되도록 만들어졌다.,1-3. 전반적으로 교육 내용이 도움이 되었다.,1-4. 본 강의를 다른 사람에게도 추천할 의향이 있다.,1-5. 기타 의견(없으시면 생략),2-1. 강의 준비 및 열의에 만족하였다.,2-2. 강의 기법과 질문 응답 등에 만족하였다.,2-3. 강의에 대한 강사의 전문성에 만족하였다.,2-4. 기타 의견(없으시면 생략),"3-1. 환경, 조명, 소음, 화면공유 등 교육 진행 환경이 만족스러웠다.",3-2. 교육운영 인력의 교육 진행과 응대는 만족스러웠다.,3-3. 기타 의견(없으시면 생략),신청기관,강의일,1-1,1-2,1-3,1-4,2-1,2-2,2-3,3-1,3-2,평균,설문인원,만족도
0,2023. 10. 4 오전 5:58:09,215002,구현모,공공기관,KOTRA,고객전략팀,,7,7,7,7,,7,7,7,,7,7,,KOTRA,2023-09-19,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.0,25,95.428
1,2023. 9. 21 오후 2:31:28,meekim@kotra.or.kr,김미연,공공기관,KOTRA,정보보안운영팀,공공SW사업 과업변경 실무,7,7,7,7,,7,7,7,,7,7,,KOTRA,2023-09-19,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.0,25,95.428
2,2023. 9. 22 오후 5:05:14,206040,김성재,공공기관,KOTRA,해외정보관리팀,,6,6,6,6,,5,5,5,,7,7,,KOTRA,2023-09-19,85.714286,85.714286,85.714286,85.714286,71.428571,71.428571,71.428571,100.0,100.0,84.1,25,95.428


In [61]:
# 2. '기타 의견' 컬럼 생성
def count_non_empty_values(row):
    return row[['1-5. 기타 의견(없으시면 생략)', '2-4. 기타 의견(없으시면 생략)', '3-3. 기타 의견(없으시면 생략)']].apply(lambda x: x != '').sum()

df['기타 의견'] = df.apply(count_non_empty_values, axis=1)

# 3. '1-5. 기타 의견(없으시면 생략)', '2-4. 기타 의견(없으시면 생략)', '3-3. 기타 의견(없으시면 생략)' 컬럼의 값을 합치기
df[['1-5. 기타 의견(없으시면 생략)', '2-4. 기타 의견(없으시면 생략)', '3-3. 기타 의견(없으시면 생략)']] = df[['1-5. 기타 의견(없으시면 생략)', '2-4. 기타 의견(없으시면 생략)', '3-3. 기타 의견(없으시면 생략)']].fillna('')
df['모든 의견'] = df[['1-5. 기타 의견(없으시면 생략)', '2-4. 기타 의견(없으시면 생략)', '3-3. 기타 의견(없으시면 생략)']].agg(', '.join, axis=1).str.replace(r'(,)\s+', '', regex=True).str.strip(', ')

# 4. '강의일'과 '신청기관' 별로 그룹화하고, '모든 의견' 컬럼의 값들을 개행 문자('\n')로 구분하여 합침
grouped = df.groupby(['강의일', '신청기관'])['모든 의견'].agg(lambda x: '\n'.join(filter(None, x))).reset_index()

# 5. 불필요한 ', ' 제거 및 '기타 의견', '만족도' 추가
grouped_final = df.groupby(['강의일', '신청기관']).agg({'기타 의견': 'sum', '만족도': 'mean'}).reset_index()
final_df = pd.merge(grouped, grouped_final, on=['강의일', '신청기관'])
final_df = final_df[['강의일', '신청기관', '만족도', '기타 의견', '모든 의견']]

final_df.head()

,강의일,신청기관,만족도,기타 의견,모든 의견
0,2023-09-14,한국교육개발원,90.135714,9,원론적인 설명보다 실용적인 설명이라 실무자입장에서 도움이 많이됨동영상 녹화 후 복습...
1,2023-09-18,KOTRA,94.216,6,강사님의 귀에 쏙쏙 들어오는 강의가 매우 좋았습니다\n실무 및 향후 감사대비 실용성...
2,2023-09-19,KOTRA,95.428,1,강사분 역량이 역대 최상급이었습니다.
3,2023-09-20,한국문화정보원,94.992308,4,법제도 및 특이사례에 대한 경험 많은 소개내용은 좋았습니다. PPT 교욱자료 화면이...
4,2023-10-05,국민연금공단,95.058491,25,강의내용이 좋았습니다\n초급자 과정도 있으면 좋을 것 같습니다\n교육자료 공유 부탁...


In [62]:
# '수강인원' 컬럼 추가 (모든 값이 null)
final_df['수강인원'] = None

# '설문인원' 컬럼 추가 (각 그룹의 행 수를 카운트하여 '설문인원'으로 간주)
survey_counts = df.groupby(['강의일', '신청기관']).size().reset_index(name='설문인원')

# 'final_df'와 'survey_counts' 병합
final_df_extended = pd.merge(final_df, survey_counts, on=['강의일', '신청기관'], how='left')

# 컬럼 순서 재정렬
final_df_extended = final_df_extended[['강의일', '신청기관', '수강인원', '설문인원', '만족도', '기타 의견', '모든 의견']]

final_df_extended.head()


,강의일,신청기관,수강인원,설문인원,만족도,기타 의견,모든 의견
0,2023-09-14,한국교육개발원,None,42,90.135714,9,원론적인 설명보다 실용적인 설명이라 실무자입장에서 도움이 많이됨동영상 녹화 후 복습...
1,2023-09-18,KOTRA,None,25,94.216,6,강사님의 귀에 쏙쏙 들어오는 강의가 매우 좋았습니다\n실무 및 향후 감사대비 실용성...
2,2023-09-19,KOTRA,None,25,95.428,1,강사분 역량이 역대 최상급이었습니다.
3,2023-09-20,한국문화정보원,None,13,94.992308,4,법제도 및 특이사례에 대한 경험 많은 소개내용은 좋았습니다. PPT 교욱자료 화면이...
4,2023-10-05,국민연금공단,None,53,95.058491,25,강의내용이 좋았습니다\n초급자 과정도 있으면 좋을 것 같습니다\n교육자료 공유 부탁...


In [63]:
from openpyxl import load_workbook
from openpyxl.styles import Alignment

# 파일 경로
file_save = '오프라인 만족도.xlsx'
save_path = 'G:/.shortcut-targets-by-id/1pm-mzTMxN90n5Axi5ATZbh2kl8wZEOlv/2023 공공SW 수발주자 역량강화 교육 프로젝트/4.사업관리/교육통계/'
full_path = save_path + file_save

# 'ExcelWriter' 객체 생성 (engine을 'openpyxl'로 설정)
with pd.ExcelWriter(full_path, engine='openpyxl') as writer:
    # 첫 번째 시트에 'df' 데이터프레임 저장
    df.to_excel(writer, sheet_name='Sheet1', index=False)

    # 세 번째 시트에 'final_df' 데이터프레임 저장
    final_df_extended.to_excel(writer, sheet_name='Sheet2', index=False)

    # 'openpyxl' 워크북 객체 가져오기
    workbook = writer.book

    # 'Sheet2'의 '모든 의견' 컬럼 셀 서식 조정
    sheet = workbook['Sheet2']
    for row in sheet.iter_rows(min_row=1, min_col=7, max_col=7):  # '모든 의견' 컬럼이 5번째라고 가정
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

# 파일 경로 반환 (실제 환경에서 파일이 저장됨)
full_path


'G:/.shortcut-targets-by-id/1pm-mzTMxN90n5Axi5ATZbh2kl8wZEOlv/2023 공공SW 수발주자 역량강화 교육 프로젝트/4.사업관리/교육통계/오프라인 만족도.xlsx'